## **Step 1 Download and Extract the Dataset**

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/JHU2022SPRING/DeepLearningProject_Erix
!ls

Mounted at /content/drive
/content/drive/MyDrive/JHU2022SPRING/DeepLearningProject_Erix
hmdb51_org  hmdb51_org.rar  model_structure_plot.png


In [2]:
!nvidia-smi

Tue May  3 22:33:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
from tqdm import tqdm
file_name='hmdb51_org.rar'
if os.path.exists(file_name):
  print(file_name, " has been already downloaded")
else:
  print(file_name, " has Not downloaded Yet, Wait......")
  os.system(f"""wget -c --read-timeout=5 --tries=0 http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar""")

!pip install unrar
!mkdir hmdb51_org
get_ipython().system_raw("unrar x hmdb51_org.rar hmdb51_org")

hmdb51_org.rar  has Not downloaded Yet, Wait......


In [3]:
import os
os.getcwd() + '/hmdb51_org/'

'/content/drive/MyDrive/JHU2022SPRING/DeepLearningProject_Erix/hmdb51_org/'

In [ ]:
ls

drive/  hmdb51_org/  hmdb51_org.rar  sample_data/


In [ ]:
import glob as g
import os
path=os.getcwd() + '/hmdb51_org/'
data_files = []
for i in g.glob( path+ '/**/*.rar',recursive=True):
    data_files.append(i)
data_files

In [ ]:
for file in tqdm(data_files):
  get_ipython().system_raw("unrar x {} hmdb51_org".format(file))
  os.remove(file)


#### **Import Required Libraries:**
Start by importing all required libraries.

In [2]:
# run
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from moviepy.editor import *
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
from math import floor

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)3063808/45929032 bytes (6.7%)7086080/45929032 bytes (15.4%)11321344/45929032 bytes (24.6%)15368192/45929032 bytes (33.5%)19439616/45929032 bytes (42.3%)23633920/45929032 bytes (51.5%)27820032/45929032 bytes (60.6%)32014336/45929032 bytes (69.7%)36200448/45929032 bytes (78.8%)40435712/45929032 bytes (88.0%)44679168/45929032 bytes (97.3%)45929032/45929032 bytes (100.0%)
  Done
File saved as /root

**Set Numpy, Python & Tensorflow seeds to get consistent results.**

In [3]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:
# # Create a Matplotlib figure
# plt.figure(figsize = (50, 50))

# # Get Names of all classes in hmdb51_org
# all_classes_names = os.listdir('hmdb51_org')

# # Generate a random sample of images each time the cell runs
# random_range = random.sample(range(len(all_classes_names)), 51)

# # Iterating through all the random samples
# for counter, random_index in enumerate(random_range, 1):

#     # Getting Class Name using Random Index
#     selected_class_Name = all_classes_names[random_index]

#     # Getting a list of all the video files present in a Class Directory
#     video_files_names_list = os.listdir(f'hmdb51_org/{selected_class_Name}')

#     # Randomly selecting a video file
#     selected_video_file_name = random.choice(video_files_names_list)

#     # Reading the Video File Using the Video Capture
#     video_reader = cv2.VideoCapture(f'hmdb51_org/{selected_class_Name}/{selected_video_file_name}')
    
#     # Reading The First Frame of the Video File
#     _, bgr_frame = video_reader.read()

#     # Closing the VideoCapture object and releasing all resources. 
#     video_reader.release()

#     # Converting the BGR Frame to RGB Frame 
#     rgb_frame = cv2.cvtColor(bgr_frame, cv2.COLOR_BGR2RGB)

#     # Adding The Class Name Text on top of the Video Frame.
#     cv2.putText(rgb_frame, selected_class_Name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
#     # Assigning the Frame to a specific position of a subplot
#     plt.subplot(11, 5, counter)
#     plt.imshow(rgb_frame)
#     plt.axis('off')

In [ ]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())
import tensorflow as tf
if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10875468119064278094
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16154099712
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8152170386231438943
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]
Default GPU Device: /device:GPU:0


In [ ]:
dataset_directory

'/content/drive/MyDrive/JHU2022SPRING/DeepLearningProject_Erix/hmdb51_org/'

Calling the **create_dataset** method which returns features and labels.

In [4]:
%%time
# erix dataprocessing edition 2, avoid ram crash when splitting tran/test..
# erix cell, get training and testing set 
image_height, image_width = 64, 64
max_images_per_class = 2000
dataset_directory = "/content/drive/MyDrive/JHU2022SPRING/DeepLearningProject_Erix/hmdb51_org/"
#classes_list = [ 'eat', 'fall_floor', 'fencing', 'flic_flac', 'golf', 'handstand', 'sword', 'sword_exercise', 'walk', 'hit', 'shoot_ball', 'shoot_bow', 'shoot_gun', 'sit', 'situp', 'smile', 'smoke', 'somersault', 'stand', 'swing_baseball', 'hug', 'jump', 'kick', 'kick_ball','kiss', 'laugh', 'pick', 'pour', 'brush_hair','cartwheel', 'catch', 'chew', 'clap', 'climb', 'climb_stairs', 'dive', 'draw_sword', 'dribble','drink', 'pullup', 'punch', 'push', 'pushup', 'ride_bike', 'ride_horse', 'run', 'shake_hands']
classes_list_all = [ 'eat', 'fall_floor', 'fencing', 'walk', 'hit', 'sit', 'situp', 'smile', 'smoke', 'stand', 'hug', 'jump', 'kick','kiss', 
                'laugh', 'pick', 'catch', 'chew', 'clap', 'climb', 'dive', 'drink', 'punch', 'push', 'ride_bike', 'run', 'shake_hands']
# 10
classes_list = ['wave', 'brush_hair', 'catch', 'climb', 'eat', 'pick', 'pour', 'smile', 'situp' , 'kiss']
classes_list_15 = ['Wave', 'brush_hair', 'catch', 'climb', 'eat', 'pick', 'pour', 'smile', 'situp' , 'kiss', 'golf', 'sword', 'hit', 'shoot_ball', 'stand']  
classes_list_20 = [] # todo? 



model_output_size = len(classes_list)

def frames_extraction(video_path):
    # Empty List declared to store video frames
    frames_list = []
    
    # Reading the Video File Using the VideoCapture
    video_reader = cv2.VideoCapture(video_path)

    # Iterating through Video Frames
    while True:

        # Reading a frame from the video file 
        success, frame = video_reader.read() 

        # If Video frame was not successfully read then break the loop
        if not success:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # Appending the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()

    # returning the frames list 
    return frames_list

# erix func
def get_frame_indices(target_len, frames_len):
    res = []
    for n_i in range(1,target_len+1):
        res.append(floor((frames_len-1)*n_i/target_len))
    return res
# erix func

# erix func
# newest update.. only use a few videos

frame_per_video = 13 # 40
def sample_videos(features, labels, num_samples=6, frame_per_video=frame_per_video):
    last_label = None
    sampled_features = np.zeros([num_samples*frame_per_video, 64, 64, 3])
    sampled_labels = []
    sample_idx = 0
    for idx, cur_label in enumerate(labels):
        if cur_label == last_label:
            continue
        last_label = cur_label
        sampled_features[sample_idx:] = features[idx]
        sampled_features[sample_idx: + frame_per_video] = features[idx:idx + frame_per_video]
        for j in frame_per_video:
            sampled_labels.append(cur_label)
    return sampled_features, sampled_labels


# erix func
def get_video_frames(min_num_frames=frame_per_video, classes_list=classes_list):

    
    # generate eatures_train, y = labels_train
    test_ratio = 0.2
    features_train = []
    labels_train = []
    features_test = []
    labels_test = []


    # Iterating through all the classes mentioned in the classes list
    # each video is an np.array of size (40, 64, 64, 3)
    # get sizes 
    train_size, test_size = 0, 0
    for class_index, class_name in enumerate(classes_list):
        files_list = os.listdir(os.path.join(dataset_directory, class_name))
        num_videos = len(files_list)
        for idx, file_name in enumerate(files_list):
            if idx <= num_videos * (1-test_ratio):
                train_size +=min_num_frames
            else:
                test_size +=min_num_frames
    features_train = np.zeros([train_size, 64, 64, 3])
    features_test = np.zeros([test_size, 64, 64, 3])
    train_size, test_size = 0, 0
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))
        frames_per_class = []
        num_videos = len(files_list) # number of videos of this class!
        for idx, file_name in enumerate(files_list):
            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)

            # Calling the frame_extraction method for every video file path
            cur_video_frames = frames_extraction(video_file_path)

            # 

            # Appending the frames to a temporary list.
            frame_indices = get_frame_indices(min_num_frames, len(cur_video_frames))
            cur_video_frames = [cur_video_frames[i] for i in frame_indices]
            cur_video_frames = np.asarray(cur_video_frames)


            if idx <= num_videos * (1-test_ratio):
                features_train[train_size:train_size+min_num_frames] = cur_video_frames
                labels_train.extend(min_num_frames*[class_name])
                train_size+=min_num_frames
            else:
                # test set
                features_test[test_size:test_size+min_num_frames] = cur_video_frames
                labels_test.extend(min_num_frames*[class_name])
                test_size+=min_num_frames
        # if class_index==2:
        #     break
        # break  !! TODO: remove this break

    return features_train, features_test, labels_train, labels_test


# erix

# TODO

# ResNet
# def erix_evaluate(features, labels, predictions min_num_frames=40, classes_list=classes_list):
    
#     for i in range(len(features//min_num_frames)):
#         if i%

# get the dictionary dataset
features_train, features_test, labels_train, labels_test = get_video_frames(min_num_frames=13, classes_list=classes_list)

Extracting Data of Class: wave
Extracting Data of Class: brush_hair
Extracting Data of Class: catch
Extracting Data of Class: climb
Extracting Data of Class: eat
Extracting Data of Class: pick
Extracting Data of Class: pour
Extracting Data of Class: smile
Extracting Data of Class: situp
Extracting Data of Class: kiss
CPU times: user 1min 7s, sys: 5.11 s, total: 1min 12s
Wall time: 1min 42s


In [ ]:
#erix cell, clear ram
import gc
gc.collect()

17949

In [5]:
# erix note:
# use train_split for splitting training srt

def sample_videos_testset(features, labels, num_samples=6, frame_per_video=40):
    last_label = None
    sampled_features = np.zeros([num_samples*frame_per_video*len(classes_list), 64, 64, 3])
    sampled_labels = []
    sample_idx = 0
    for idx, cur_label in enumerate(labels):
        if cur_label == last_label:
            continue
        # print(idx)
        # print(sample_idx)
        last_label = cur_label
        # sampled_features[sample_idx:] = features[idx]
        sampled_features[sample_idx: sample_idx+ num_samples*frame_per_video] = features[idx:idx + num_samples*frame_per_video]
        for j in range(num_samples*frame_per_video):
            sampled_labels.append(cur_label)
        sample_idx += num_samples*frame_per_video
    return sampled_features, sampled_labels
sampled_features, sampled_labels = sample_videos_testset(features_test, 
                                                         labels_test, num_samples=6, 
                                                         frame_per_video=13)

In [ ]:
print(len(sampled_labels)/frame_per_video) # num videos
print(len(sampled_labels)/frame_per_video/model_output_size)# num videos per class

108.0
4.0


In [ ]:
# verify , some number for 27 classes
print("num of frames:")
print(len(labels_train))
print(len(labels_test))

print("num of videos:")
print(len(labels_train)/frame_per_video)
print(len(labels_test)/frame_per_video)

len(features_train) == len(labels_train)

num of frames:
40339
9854
num of videos:
3103.0
758.0


True

In [ ]:
model_output_size

27

In [ ]:
# Let's create a function that will construct our model
def create_model():

    # We will use a Sequential model for model construction
    model = Sequential()

    # Defining The Model Architecture
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(model_output_size, activation = 'softmax'))

    # Printing the models summary
    model.summary()

    return model


# Calling the create_model method
model = create_model()

print("Model Created Successfully!")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 conv2d_3 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization_2 (Batc  (None, 60, 60, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 global_average_pooling2d_1   (None, 64)               0         
 (GlobalAveragePooling2D)                                        
                                                      

In [ ]:
# plot_model(model, to_file = 'model_structure_plot.png', show_shapes = True, show_layer_names = True)

In [ ]:
len(labels_train)

40339

In [ ]:
import gc
gc.collect()
from keras import backend as K
K.clear_session()

In [12]:
# erix cell
# generate labels, and shuffle the training set

class_idx_map = {}
# map classnames to idx 
for idx, c in enumerate(classes_list):
    class_idx_map[c] = idx


# Erix added . Training
class_idx_map = {}
# convert 
for idx, c in enumerate(classes_list):
    class_idx_map[c] = idx
one_hot_encoded_erix_labels_train = to_categorical(list(map(lambda x: class_idx_map[x], labels_train)))
one_hot_encoded_labels_test = to_categorical(list(map(lambda x: class_idx_map[x], labels_test)))

# selected test samples
one_hot_encoded_labels_sampled_test = to_categorical(list(map(lambda x: class_idx_map[x], sampled_labels)))

# shuffle the training set!
# there's a problem with keras shuffle! 
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

features_train_shuffle, one_hot_encoded_erix_labels_train_shuffle = unison_shuffled_copies(features_train, one_hot_encoded_erix_labels_train)


In [ ]:
# erix cell, Baseline Model
import gc
gc.collect()
# would crash.. if i have 40 frames per video (120k frames in total)
# thus later trained on 13 frames per video.
# shuffle is not ..random enough
model = create_model()

seed_constant = 44
np.random.seed(seed_constant)
random.seed(seed_constant+1)
tf.random.set_seed(seed_constant+2)

# manually shuffle the training set! model.fit(shuffle=?) is problematic.

# one_hot_encoded_erix_labels_train = to_categorical(list(map(lambda x: class_idx_map[x], labels_train)))
shuffled=True
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
# Start Training
model_training_history = model.fit(x = features_train_shuffle, y = one_hot_encoded_erix_labels_train_shuffle, epochs = 200, 
                                   batch_size = 15 , shuffle = True, validation_split = 0.2)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 60, 60, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 64)               0         
 lobalAveragePooling2D)                                          
                                                        

KeyboardInterrupt: ignored

In [ ]:
# erix added, evaluation(baseline):
model_evaluation_history = model.evaluate(features_test, one_hot_encoded_labels_test)

84/84 [==============================] - 0s 4ms/step - loss: 4.8858 - accuracy: 0.4439


In [22]:
# Erix vgg (Ken's codes)

from keras.applications.vgg16 import VGG16
# load model

...
# load model without output layer
model = VGG16(include_top=False)
model.trainable=False # True # False

import keras
inputs = keras.Input(shape=(64, 64, 3))
x = model(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="my_model")

opt = tf.keras.optimizers.Adam(learning_rate=0.005) # erix note: adjusted this
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, None, None, 512)   14714688  
                                                                 
 global_average_pooling2d_5   (None, 512)              0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 10)                5130      
                                                                 
Total params: 14,719,818
Trainable params: 5,130
Non-trainable params: 14,714,688
_________________________________________________________________


In [20]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

# Start Training
# note batch size is 32.
model_training_history_resnet = model.fit(x = features_train_shuffle, y = one_hot_encoded_erix_labels_train_shuffle, epochs = 200, 
                                   batch_size = 32 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])


Epoch 1/200
275/275 [==============================] - 5s 17ms/step - loss: 1.0854 - accuracy: 0.6438 - val_loss: 0.8010 - val_accuracy: 0.7319
Epoch 2/200
275/275 [==============================] - 4s 14ms/step - loss: 0.6480 - accuracy: 0.8001 - val_loss: 0.6169 - val_accuracy: 0.7915
Epoch 3/200
275/275 [==============================] - 4s 14ms/step - loss: 0.4992 - accuracy: 0.8534 - val_loss: 0.5271 - val_accuracy: 0.8361
Epoch 4/200
275/275 [==============================] - 4s 14ms/step - loss: 0.4276 - accuracy: 0.8727 - val_loss: 0.4816 - val_accuracy: 0.8471
Epoch 5/200
275/275 [==============================] - 4s 14ms/step - loss: 0.3685 - accuracy: 0.8954 - val_loss: 0.4306 - val_accuracy: 0.8621
Epoch 6/200
275/275 [==============================] - 4s 14ms/step - loss: 0.3275 - accuracy: 0.9052 - val_loss: 0.3829 - val_accuracy: 0.8844
Epoch 7/200
275/275 [==============================] - 4s 14ms/step - loss: 0.3007 - accuracy: 0.9162 - val_loss: 0.3543 - val_accuracy:

In [21]:
# erix added, evaluation:
model_evaluation_history_resnet = model.evaluate(features_test, one_hot_encoded_labels_test)


84/84 [==============================] - 1s 12ms/step - loss: 2.6967 - accuracy: 0.4394


In [23]:
# lower learning rate

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

# Start Training
# note batch size is 32.
model_training_history_resnet = model.fit(x = features_train_shuffle, y = one_hot_encoded_erix_labels_train_shuffle, epochs = 200, 
                                   batch_size = 32 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])
# erix added, evaluation:
model_evaluation_history_resnet = model.evaluate(features_test, one_hot_encoded_labels_test)


Epoch 1/200
275/275 [==============================] - 5s 16ms/step - loss: 1.2291 - accuracy: 0.6059 - val_loss: 0.9099 - val_accuracy: 0.7119
Epoch 2/200
275/275 [==============================] - 4s 14ms/step - loss: 0.7930 - accuracy: 0.7607 - val_loss: 0.7203 - val_accuracy: 0.7715
Epoch 3/200
275/275 [==============================] - 4s 14ms/step - loss: 0.6400 - accuracy: 0.8080 - val_loss: 0.6234 - val_accuracy: 0.8152
Epoch 4/200
275/275 [==============================] - 4s 14ms/step - loss: 0.5493 - accuracy: 0.8426 - val_loss: 0.5562 - val_accuracy: 0.8330
Epoch 5/200
275/275 [==============================] - 4s 14ms/step - loss: 0.4856 - accuracy: 0.8631 - val_loss: 0.5370 - val_accuracy: 0.8261
Epoch 6/200
275/275 [==============================] - 4s 14ms/step - loss: 0.4368 - accuracy: 0.8754 - val_loss: 0.4769 - val_accuracy: 0.8557
Epoch 7/200
275/275 [==============================] - 4s 14ms/step - loss: 0.3996 - accuracy: 0.8877 - val_loss: 0.4380 - val_accuracy:



Previous: 

the old dataset: 2000 frames for each class (27*2000 = 54k frames)
then: split training & testing set

problem: frames in train&test sets can be from the same videos




split videos -> training & testing videos

train the model on : only the frames in training-videos

true test set: frames from unseen videos



# 
40 - ~200 frames per video
13 frames per video ()

# 
reducing the number of classes 27 -> 10?



# 

[ 13 frames for each video ] ( for LSTM? )


1. frame dataset: each training sample is a frame.
    ### video label will be the most common predicted label for 13 frames.

2. list_of_frame dataset: each training sample is of 13 frames (or any other number)


In [ ]:
# Erix added, Ken's model
import keras

resnet_50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
resnet_50.trainable=True

inputs = keras.Input(shape=(64, 64, 3))
x = resnet_50(inputs)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="my_model")


opt = tf.keras.optimizers.Adam(learning_rate=0.0001) # erix note: adjusted this
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 resnet50 (Functional)       (None, None, None, 2048)  23587712  
                                                                 
 global_average_pooling2d_2   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_2 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
# Erix added, Ken's model, training

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, mode = 'min', restore_best_weights = True)

# Start Training
# note batch size is 15.
model_training_history_resnet = model.fit(x = features_train_shuffle, y = one_hot_encoded_erix_labels_train_shuffle, epochs = 200, 
                                   batch_size = 15 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])


Epoch 1/200
586/586 [==============================] - 30s 42ms/step - loss: 0.5037 - accuracy: 0.8456 - val_loss: 3.8204 - val_accuracy: 0.1220
Epoch 2/200
586/586 [==============================] - 23s 40ms/step - loss: 0.0650 - accuracy: 0.9836 - val_loss: 0.0899 - val_accuracy: 0.9795
Epoch 3/200
586/586 [==============================] - 23s 40ms/step - loss: 0.0560 - accuracy: 0.9841 - val_loss: 0.0646 - val_accuracy: 0.9841
Epoch 4/200
586/586 [==============================] - 23s 39ms/step - loss: 0.0495 - accuracy: 0.9863 - val_loss: 0.0749 - val_accuracy: 0.9804
Epoch 5/200
586/586 [==============================] - 22s 38ms/step - loss: 0.0441 - accuracy: 0.9863 - val_loss: 0.0604 - val_accuracy: 0.9868
Epoch 6/200
586/586 [==============================] - 22s 38ms/step - loss: 0.0292 - accuracy: 0.9915 - val_loss: 0.0671 - val_accuracy: 0.9863
Epoch 7/200
586/586 [==============================] - 23s 39ms/step - loss: 0.0220 - accuracy: 0.9942 - val_loss: 0.0649 - val_ac

In [ ]:
# erix added, evaluation:
model_evaluation_history_resnet = model.evaluate(features_test, one_hot_encoded_labels_test)

84/84 [==============================] - 2s 18ms/step - loss: 2.0893 - accuracy: 0.5182


#The rest is not important - Erix noted

Erix dataset processing:
Problem:  RAM crashes easily!

My goal: split new training and test datasets s.t. we have a list of frames for a video as a single sample. 

How: Firstly, I split training videos and test videos. For each video, I extract some number of frames (Currently using 13 frames/video)

Problem with the original notebook design: frames from the same video can appear in both training&test set

New evaluation method: Firstly, we'll evaluate only on the frames of the unseen videos. There's 13 frames per video. 
IF WE ONLY CLASSIFY FRAMES,we'll predict 13 labels for 13 frames and return the most common label for each video.
IF the input of the model is a list of frames(13 frames), then just use the model's output.



In [ ]:
# erix notes: experimenting training on small dataset

# 4*27*40/5*.8(validation costs 20%) = 691 batches per epoch~

class_idx_map = {}
# convert 
for idx, c in enumerate(classes_list):
    class_idx_map[c] = idx
# sampled_features
one_hot_encoded_sampled_labels = to_categorical(list(map(lambda x: class_idx_map[x], sampled_labels)))

model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start Training
model_training_history = model.fit(x = sampled_features, y = one_hot_encoded_sampled_labels, epochs = 200, batch_size = 5 , shuffle = True, validation_split = 0.2)


Epoch 1/200
692/692 [==============================] - 18s 6ms/step - loss: 1.8633 - accuracy: 0.4586 - val_loss: 7.6005 - val_accuracy: 0.0000e+00
Epoch 2/200
692/692 [==============================] - 4s 5ms/step - loss: 0.8153 - accuracy: 0.7578 - val_loss: 8.5306 - val_accuracy: 0.0741
Epoch 3/200
692/692 [==============================] - 4s 6ms/step - loss: 0.4869 - accuracy: 0.8478 - val_loss: 11.6451 - val_accuracy: 0.0741
Epoch 4/200
692/692 [==============================] - 4s 6ms/step - loss: 0.2644 - accuracy: 0.9216 - val_loss: 11.0224 - val_accuracy: 0.0556
Epoch 5/200
692/692 [==============================] - 4s 5ms/step - loss: 0.2212 - accuracy: 0.9410 - val_loss: 10.5187 - val_accuracy: 0.0498
Epoch 6/200
692/692 [==============================] - 4s 5ms/step - loss: 0.2008 - accuracy: 0.9433 - val_loss: 13.1875 - val_accuracy: 0.0741
Epoch 7/200
692/692 [==============================] - 4s 5ms/step - loss: 0.1549 - accuracy: 0.9543 - val_loss: 11.6808 - val_accura

KeyboardInterrupt: ignored

In [ ]:
def sample_training_frames(features_train, one_hot_labels_train):

    _,__, labels_train_kept, labels_train_kept = train_test_split(features_train, 
                                                                                one_hot_labels_train, test_size = 0.05, shuffle = True)
    return labels_train_kept, labels_train_kept

labels_train_kept, labels_train_kept = sample_training_frames(features_train, one_hot_encoded_labels_train)

In [ ]:
%%time

# Erix added . Training
class_idx_map = {}
# convert 
for idx, c in enumerate(classes_list):
    class_idx_map[c] = idx
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels_train = to_categorical(list(map(lambda x: class_idx_map[x], labels_train)))
one_hot_encoded_labels_test = to_categorical(list(map(lambda x: class_idx_map[x], labels_test)))
# erix  edited
# Adding the Early Stopping Callback to the model which will continuously monitor the validation loss metric for every epoch.
# If the models validation loss does not decrease after 15 consecutive epochs, the training will be stopped and the weight which reported the lowest validation loss will be retored in the model.
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start Training
model_training_history = model.fit(x = features_train, y = one_hot_encoded_labels_train, epochs = 200, batch_size = 5 , shuffle = True, validation_split = 0.2)
#model_training_history = model.fit(x = features_train, y = labels_train, epochs = 200, batch_size = 5 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [ ]:
%%time
# Adding the Early Stopping Callback to the model which will continuously monitor the validation loss metric for every epoch.
# If the models validation loss does not decrease after 15 consecutive epochs, the training will be stopped and the weight which reported the lowest validation loss will be retored in the model.
#early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)

# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])

# Start Training
model_training_history = model.fit(x = features_train, y = labels_train, epochs = 200, batch_size = 5 , shuffle = True, validation_split = 0.2)
#model_training_history = model.fit(x = features_train, y = labels_train, epochs = 200, batch_size = 5 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/200
6912/6912 [==============================] - 38s 4ms/step - loss: 2.8862 - accuracy: 0.1686 - val_loss: 2.8645 - val_accuracy: 0.2303
Epoch 2/200
6912/6912 [==============================] - 27s 4ms/step - loss: 2.4793 - accuracy: 0.2736 - val_loss: 2.5381 - val_accuracy: 0.3201
Epoch 3/200
6912/6912 [==============================] - 27s 4ms/step - loss: 2.2475 - accuracy: 0.3463 - val_loss: 2.4518 - val_accuracy: 0.3713
Epoch 4/200
6912/6912 [==============================] - 27s 4ms/step - loss: 2.0733 - accuracy: 0.3980 - val_loss: 2.3936 - val_accuracy: 0.4046
Epoch 5/200
6912/6912 [==============================] - 26s 4ms/step - loss: 1.9567 - accuracy: 0.4301 - val_loss: 2.7502 - val_accuracy: 0.3936
Epoch 6/200
6912/6912 [==============================] - 27s 4ms/step - loss: 1.8505 - accuracy: 0.4642 - val_loss: 2.1568 - val_accuracy: 0.4999
Epoch 7/200
6912/6912 [==============================] - 27s 4ms/step - loss: 1.7690 - accuracy: 0.4882 - val_loss: 2.4112 -

KeyboardInterrupt: ignored

In [ ]:
model_evaluation_history = model.evaluate(features_test, one_hot_encoded_labels_test)

308/308 [==============================] - 2s 6ms/step - loss: 4.5731 - accuracy: 0.1988


In [ ]:
y_pred = model.predict(features_test)

KeyboardInterrupt: ignored

In [ ]:
y_pred=np.argmax(y_pred, axis=1)
labels_test=np.argmax(labels_test, axis=1)
print(classification_report(labels_test,y_pred,target_names=classes_list))

In [ ]:
cm = confusion_matrix(labels_test,y_pred)

In [ ]:
print('Train Accuracy of Model is {} and Test Accuracy of Model is {}'.format(model_training_history.history['accuracy'][-1],model_evaluation_history[1]))

In [ ]:
import seaborn as sns
plt.figure(figsize=[7,6])
sns.heatmap(cm,cmap="Reds", annot=True, fmt='.0f')
plt.show()

In [ ]:
labels_test

array([16,  7,  8, ...,  1,  0, 20])

In [ ]:
def plot_metric(metric_name_1, metric_name_2, plot_name):
    # Get Metric values using metric names as identifiers
    metric_value_1 = model_training_history.history[metric_name_1]
    metric_value_2 = model_training_history.history[metric_name_2]

    # Constructing a range object which will be used as time 
    epochs = range(len(metric_value_1))
  
    # Plotting the Graph
    plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
    plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)
  
    # Adding title to the plot
    plt.title(str(plot_name))
    #Saving the Plot
    plt.savefig(str(plot_name)+".png")
    # Adding legend to the plot
    plt.legend()

In [ ]:
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

Make predictions

In [ ]:
def predict_on_uploaded_video(video_file_path, output_file_path, window_size):


    predicted_labels_probabilities_deque = deque(maxlen = window_size)

    # Reading the Video File using the VideoCapture Object
    video_reader = cv2.VideoCapture(video_file_path)

    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True: 

        # Reading The Frame
        status, frame = video_reader.read() 

        if not status:
            break

        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255

        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]

        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:

            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)

            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)

            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]
          
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Writing The Frame
        video_writer.write(frame)
    
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
predict_on_uploaded_video("#########.avi",1)